In [74]:
import numpy as np
import pandas as pd

startsim = 101
endsim = 105

In [75]:
basedir = '/scratch/sbanerj/quasi_laplace_gwas/simulated_phenotype/meta_nold'
jobsdir = '/usr/users/sbanerj/quasi_laplace_gwas/simulation/jobsubs/meta_nold_bimodal'

In [76]:
def get_resource_from_lsfout(filename, resource_name = 'Run time'):
    with open (filename, 'r') as mfile:
        resource_block = False
        while not resource_block:
            mline = mfile.readline().strip()
            if mline.startswith('Resource usage summary:'):
                resource_block = True

        resource_found = False
        while not resource_found:
            mline = mfile.readline().strip()
            if mline.startswith(resource_name):
                resource_found = True
                resource = mline.split(':')[1].strip().split()[0].strip()
                resource = float(resource)
    return resource

In [112]:
import os

zmax = 4
muvar = '0'
nfeat = 112
blore_nofeat_times = list()
blore_feat_times = list()
paintor_nofeat_times = list()
paintor_feat_times = list()
finemap_times = list()
caviarbf_times = list()
resource_name = 'Max Memory'

s = 0
for sim in range(startsim, endsim+1):
    
    simname = 'sim{:03d}'.format(sim)
    simdir = os.path.join(basedir, 'simulations', simname)
    
    # B-LORE
    jobsubdir = os.path.join(jobsdir, simname, 'blore')
    all_files = os.listdir(jobsubdir)
    filelist = [x for x in all_files if x.startswith('blore_meta_{:03d}'.format(sim)) and x.endswith('{:d}_mu{:s}.out'.format(zmax, muvar))]
    if len(filelist) == 1:
        filename = os.path.join(jobsubdir, filelist[0])
        time = get_resource_from_lsfout(filename, resource_name = resource_name)
        blore_nofeat_times.append(time)
    else:
        print("Number of output files != 1")
        blore_nofeat_times.append(1000000)
        
    jobsubdir = os.path.join(jobsdir, simname, 'blore')
    all_files = os.listdir(jobsubdir)
    filelist = [x for x in all_files if x.startswith('blore_meta_feat_{:03d}'.format(sim)) and x.endswith('{:d}_mu{:s}.out'.format(zmax, muvar))]
    if len(filelist) == 1:
        filename = os.path.join(jobsubdir, filelist[0])
        time = get_resource_from_lsfout(filename, resource_name = resource_name)
        blore_feat_times.append(time)
    else:
        print("Number of output files != 1")
        blore_feat_times.append(1000000)
        
    # PAINTOR
    jobsubdir = os.path.join(jobsdir, simname, 'paintor')
    all_files = os.listdir(jobsubdir)
    filelist = [x for x in all_files if x.startswith('paintor_{:03d}'.format(sim)) and x.endswith('.out')]
    if len(filelist) == 1:
        filename = os.path.join(jobsubdir, filelist[0])
        time = get_resource_from_lsfout(filename, resource_name = resource_name)
        paintor_nofeat_times.append(time)
    else:
        print("Number of output files != 1")
        paintor_nofeat_times.append(1000000)
    
    feat_times = list()
    filelist = [x for x in all_files if x.startswith('pfa_{:03d}'.format(sim)) and x.endswith('.out')]
    if len(filelist) == nfeat + 1:
        for thisfile in filelist:
            filename = os.path.join(jobsubdir, thisfile)
            feat_times.append(get_resource_from_lsfout(filename, resource_name = resource_name))
    filelist = [x for x in all_files if x.startswith('paintor_FA_{:03d}'.format(sim)) and x.endswith('.out')]
    if sum(feat_times) > 0 and len(filelist) == 1:
        filename = os.path.join(jobsubdir, filelist[0])
        comtime = get_resource_from_lsfout(filename, resource_name = resource_name)
        paintor_feat_times.append(max(comtime, max(feat_times)))
        #paintor_feat_times.append(time + sum(feat_times))
    else:
        print("Number of output files != 1")
        paintor_feat_times.append(1000000)
        
    # FINEMAP
    jobsubdir = os.path.join(jobsdir, simname, 'finemap')
    all_files = os.listdir(jobsubdir)
    filelist = [x for x in all_files if x.startswith('finemap_c{:d}_{:03d}'.format(zmax, sim)) and x.endswith('.out')]
    if len(filelist) == 1:
        filename = os.path.join(jobsubdir, filelist[0])
        time = get_resource_from_lsfout(filename, resource_name = resource_name)
        finemap_times.append(time)
        
    # FINEMAP
    jobsubdir = os.path.join(jobsdir, simname, 'caviarbf')
    all_files = os.listdir(jobsubdir)
    filelist = [x for x in all_files if x.startswith('caviarbf_c{:d}_{:03d}'.format(2, sim)) and x.endswith('.out')]
    if len(filelist) == 1:
        filename = os.path.join(jobsubdir, filelist[0])
        time = get_resource_from_lsfout(filename, resource_name = resource_name)
        caviarbf_times.append(time)
    
        
            
    print(simname, blore_nofeat_times[s], blore_feat_times[s], paintor_nofeat_times[s], paintor_feat_times[s], finemap_times[s], caviarbf_times[s])
    s += 1

sim101 3967.0 2731.0 79.0 100.0 70.0 7.0
sim102 8353.0 1251.0 79.0 100.0 81.0 7.0
sim103 6523.0 1120.0 79.0 100.0 70.0 15.0
sim104 4610.0 633.0 79.0 100.0 55.0 7.0
sim105 2433.0 625.0 79.0 100.0 51.0 15.0


In [123]:
import os

resource_name = 'Run time'
jobsdir = '/usr/users/sbanerj/quasi_laplace_gwas/simulation/jobsubs/meta_nold_fixed'

s = 0
for sim in [1, 2, 3, 4, 5]:
    
    simname = 'sim{:03d}'.format(sim)
    simdir = os.path.join(basedir, 'simulations', simname)
    
    # B-LORE
    blore_times = list()
    jobsubdir = os.path.join(jobsdir, simname, 'blore')
    all_files = os.listdir(jobsubdir)
    filelist = [x for x in all_files if x.startswith('blore_{:d}'.format(sim)) and x.endswith('.out')]
    for thisfile in filelist:
        filename = os.path.join(jobsubdir, thisfile)
        time = get_resource_from_lsfout(filename, resource_name = resource_name)
        blore_times.append(time)
            
    print(simname, blore_times)

sim001 [688.0, 1304.0, 595.0, 785.0, 624.0]
sim002 [786.0, 585.0, 664.0, 1316.0, 676.0]
sim003 [573.0, 778.0, 575.0, 711.0, 1255.0]
sim004 [734.0, 1332.0, 714.0, 818.0, 593.0]
sim005 [636.0, 876.0, 763.0, 1283.0, 680.0]


In [124]:
jobsdir = '/usr/users/sbanerj/quasi_laplace_gwas/simulation/jobsubs/meta_nold_bimodal'

s = 0
for sim in [101, 102, 103, 104, 105]:
    
    simname = 'sim{:03d}'.format(sim)
    simdir = os.path.join(basedir, 'simulations', simname)
    
    # B-LORE
    blore_times = list()
    jobsubdir = os.path.join(jobsdir, simname, 'blore')
    all_files = os.listdir(jobsubdir)
    filelist = [x for x in all_files if x.startswith('blore_{:d}'.format(sim)) and x.endswith('.out')]
    for thisfile in filelist:
        filename = os.path.join(jobsubdir, thisfile)
        time = get_resource_from_lsfout(filename, resource_name = resource_name)
        blore_times.append(time)
            
    print(simname, blore_times)

sim101 [1356.0, 706.0, 588.0, 942.0, 941.0]
sim102 [1268.0, 698.0, 714.0, 526.0, 828.0]
sim103 [771.0, 1351.0, 1093.0, 664.0, 787.0]
sim104 [742.0, 531.0, 784.0, 691.0, 1249.0]
sim105 [1128.0, 586.0, 835.0, 1281.0, 778.0]


In [125]:
resource_name = 'Run time'
jobsdir = '/usr/users/sbanerj/quasi_laplace_gwas/simulation/jobsubs/meta_nold_random'

s = 0
for sim in [71, 72, 73, 74, 75]:
    
    simname = 'sim{:03d}'.format(sim)
    simdir = os.path.join(basedir, 'simulations', simname)
    
    # B-LORE
    blore_times = list()
    jobsubdir = os.path.join(jobsdir, simname, 'blore')
    all_files = os.listdir(jobsubdir)
    filelist = [x for x in all_files if x.startswith('blore_{:d}'.format(sim)) and x.endswith('.out')]
    for thisfile in filelist:
        filename = os.path.join(jobsubdir, thisfile)
        time = get_resource_from_lsfout(filename, resource_name = resource_name)
        blore_times.append(time)
            
    print(simname, blore_times)

sim071 [2620.0, 589.0, 789.0, 696.0, 1812.0]
sim072 [1525.0, 615.0, 705.0, 1780.0, 807.0]
sim073 [629.0, 832.0, 1424.0, 1192.0, 1605.0]
sim074 [1815.0, 957.0, 759.0, 1322.0, 649.0]
sim075 [1529.0, 951.0, 1236.0, 580.0, 1486.0]
